## Projeto de Análise de Dados e Automação

### Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
from cycler import cycler
from datetime import datetime, timedelta

yf.pdr_override()

### Analisando o Índice IBOVESPA

- O índice IBOVESPA é o mais importante indicador do desempenho médio das cotações negociadas na bolsa de valores B3.<br>
- Ele é formado pelos ativos (ações) com maior volume negociado nos últimos meses.

### Período

In [ ]:
dias = int(input('Digite o período em número de dias. '))

In [ ]:
data_inicial = datetime.now() - timedelta(days=dias)
data_final = datetime.now()

### Requisitando os Dados dos Ativos

In [ ]:
# requisiçao
df_ibov = pdr.get_data_yahoo('^BVSP', data_inicial, data_final)['Adj Close']
df_ibov

### Calculando o Retorno

In [ ]:
# localizando o último e o primeiro preço
retorno = df_ibov.iloc[-1] / df_ibov.iloc[0] - 1
print('O retorno foi de: {:.2%}'.format(retorno))

### Média Móvel

- **Média Móvel** é um dos indicadores que são observados pelos analistas e investidores que atuam no mercado financeiro, serve para verificar tendências dentro de uma análise gráfica.

In [ ]:
# Gráfico
df_ibov.plot(label='IBOV', figsize=(15, 5), color='#091D40')

# Verificando a janela do tempo
media_movel = df_ibov.rolling(30).mean()

# Gráfico
media_movel.plot(label='MM30', color='#E28A7F')

# Exibindo a legenda
plt.legend()

# Exibindo o gráfico
plt.box(False)
plt.xticks(rotation=0)
plt.tick_params(axis='x', length=0)
plt.tick_params(axis='y', length=0)
plt.title('Análise do Índice IBOVESPA x Média Móvel', fontsize=15, fontweight='bold')
plt.ylabel('Preços em (R$)')
plt.show()

In [ ]:
# Estilo
plt.style.use('dark_background')

# Gráfico
df_ibov.plot(label='IBOV', figsize=(15, 5))

# Verificando a janela do tempo
media_movel = df_ibov.rolling(30).mean()

# Gráfico
media_movel.plot(label='MM30')

# Exibindo a legenda
plt.legend()

# Exibindo o gráfico
plt.box(False)
plt.xticks(rotation=0)
plt.tick_params(axis='x', length=0)
plt.tick_params(axis='y', length=0)
plt.title('Análise do Índice IBOVESPA x Média Móvel', fontsize=15, fontweight='bold')
plt.ylabel('Preços em (R$)')
plt.show()

In [ ]:
# Cotação Máxima
cotacao_maxima = np.amax(df_ibov)
print(f'A cotação máxima do índice IBOVESPA no período foi de: {cotacao_maxima:,.0f}')

In [ ]:
# Cotação Mínima
cotacao_minima = np.amin(df_ibov)
print(f'A cotação mínima do índice IBOVESPA no período foi de: {cotacao_minima:,.0f}')

In [ ]:
cotacao_atual = df_ibov.iloc[-1]
print(f'A cotação atual do índice IBOVESPA é de: {cotacao_atual:,.0f}')

### Carregando os Dados da Carteira

In [ ]:
df_carteira = pd.read_excel('carteira.xlsx')
df_carteira

In [ ]:
dias = int(input('Digite o período em número de dias. '))

In [ ]:
data_inical = datetime.now() - timedelta(days=dias)
data_final = datetime.now()

# transformando a coluna Ticker em uma lista python
lista_ativos = list(df_carteira['Ticker'].astype(str) + '.SA')

# requisição
df_cotacoes = pdr.get_data_yahoo(lista_ativos, data_inical, data_final)['Adj Close']
df_cotacoes = round(df_cotacoes, 2)
df_cotacoes

### Tratamento de Dados

In [ ]:
# trazendo informações do dataframe
df_cotacoes.info()

- Os dados dentro do dataframe que aparecerem em branco serão tratados e preenchidos com os dados do dia anterior, porque para o caso especifico de preços de ações não seria coerente aplicar estatísticas como (moda ou média) dos valores analisados no perídodo.

In [ ]:
# preencehdo valores vázios
df_cotacoes = df_cotacoes.ffill()

In [ ]:
df_cotacoes.info()

### Normalizando os Dados

- Colocando os dados para incializarem a partir do mesmo valor (1) e estarão todos na mesma base, e se verificará que determinados ativos ou terão subido ou terão caido.<br>
- Pegando o último valor de uma coluna e dividindo pelo primeiro valor da mesma coluna, os outros valores do range da coluna será a comparação entre eles e o valor inicial.

In [ ]:
# Criando um dataframe com os dados normalizados
df_cotacoesNorm = df_cotacoes / df_cotacoes.iloc[0]

### Plotando Gráfico

In [ ]:
# Estilo
plt.style.use('dark_background')

df_cotacoesNorm.plot(figsize=(15, 5))
plt.legend(loc='upper left')
plt.xticks(rotation=0)
plt.tick_params(axis='x', length=0)
plt.tick_params(axis='y', length=0)
plt.title('Carteira Normalizada', fontsize=15, fontweight='bold')
plt.box(False)
plt.show()

### Calculando o Retorno da Carteira

**observação:** Para calcular o retorno da carteira não pode ser todo consolidado, porque precisamo calcular o retorno de cada ativo devido as cotações dos preços serem distintas.<br>
- Pega o valor de cotação de cada ativo e multiplica pela quantidade de cotas de cada um dos ativos.

### Criando um Novo Dataframe de Valores Investidos

In [ ]:
# criando um dataframe vázio
df_valorInvestido = pd.DataFrame()

# percorrendo cada ativo do dataframe
for ativo in df_carteira['Ticker']:
    cotas = df_carteira.loc[df_carteira['Ticker'] == ativo, 'Cotas'].values[0]
    # criando uma nova coluna com os valores de cada ativo
    df_valorInvestido[ativo] = cotas * df_cotacoes[f'{ativo}.SA']
    
# criando uma nova coluna com o valor total
df_valorInvestido['Total'] = df_valorInvestido.sum(axis=1)   # somando todos os valores das colunas

df_valorInvestido = round(df_valorInvestido, 2)
df_valorInvestido

### Comparando o Retorno da Carteira com o Índice do IBOVESPA

#### Normalizando os dados da Carteira e os dados do IBOVESPA

- Colocando os dados para incializarem a partir do mesmo valor (1) e estarão todos na mesma base, e se verificará que determinados ativos ou terão subido ou terão caido.<br>
- Pegando o último valor de uma coluna e dividindo pelo primeiro valor da mesma coluna, os outros valores do range da coluna será a comparação entre eles e o valor inicial.

In [ ]:
# criando as duas tabelas normalizadas
df_ibovNorm = df_ibov / df_ibov.iloc[0]

In [ ]:
# carteira normalizado
df_valorInvestidonorm = df_valorInvestido / df_valorInvestido.iloc[0]

In [ ]:
df_ibovNorm.head()

### Plotando um Gráfico do Histórico

In [ ]:
# Estilo
plt.style.use('dark_background')

df_valorInvestidonorm['Total'].plot(figsize=(15, 5), label='Carteira')
df_ibovNorm.plot(label='IBOV')

plt.legend()
plt.xticks(rotation=0)
plt.tick_params(axis='x', length=0)
plt.tick_params(axis='y', length=0)
plt.title('Retorno da Carteira vs IBOVESPA', fontsize=15, fontweight='bold')
plt.box(False)

plt.show()

### Calculando o Retorno e a Correlação

In [ ]:
# retorno IBOVESPA
retorno_ibov = df_ibovNorm[-1] -1 # o cálculo aqui foi feito direto porque a dataframe já está normalizado

# retorno da Carteira
retorno_carteira = df_valorInvestidonorm['Total'][-1] -1 

# imprimindo o retorno
print(f'O retorno do IBOV foi de: {retorno_ibov:.2%}')
print(f'O retorno da Carteira de Investimentos foi de: {retorno_carteira:.2%}')

In [ ]:
# correlação
correlacao = df_valorInvestido['Total'].corr(df_ibov)

print(f'A correlação da Carteira de Investimentos e o IBOV foi de: {correlacao:.2%}')

## Fim